<a href="https://colab.research.google.com/github/yuukis/shingenpy_doc_20250202/blob/main/20250202_Chapter4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. データ分析・可視化

（メモ）
・相関しているか


In [1]:
!pip install python-dotenv pandas

In [5]:
import os
import requests
import base64
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display

# .envファイルから環境変数を読み込む
load_dotenv("api.env")
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

# アクセストークンを取得する関数
def get_token(client_id, client_secret):
  url = "https://idm.dataplatform-yamanashi.jp/auth/realms/Smartcity/protocol/openid-connect/token"
  payload = 'grant_type=client_credentials'
  b64encoded = base64.b64encode(f"{client_id}:{client_secret}".encode("utf-8"))
  authorization = f"Basic {b64encoded.decode('utf-8')}"
  headers = {
    'Authorization': authorization,
    'Content-Type': 'application/x-www-form-urlencoded',
  }
  response = requests.request("POST", url, headers=headers, data=payload)
  accesss_token = response.json()['access_token']

  return accesss_token

def request_data(access_token, params=None):
  orion_url = "https://api.dataplatform-yamanashi.jp/ngsi/v2/entities"
  headers = {
    'Authorization': f"Bearer {access_token}"
  }
  try:
    # Send the GET request to the Orion Context Broker
    response = requests.get(orion_url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()
  except requests.exceptions.RequestException as e:
    print(f"Error fetching data from Orion Context Broker: {e}")
    return []

def convert_to_dataframe(data):
  try:
    # Extract relevant data (assuming the entities are in a list)
    entities = []
    for entity in data:
      # Flatten the attributes into a single dictionary per entity
      entity_data = {"id": entity.get("id"), "type": entity.get("type")}
      for key, value in entity.items():
        if key not in ["id", "type"]:
          # Handle attribute value (considering it's a structured object)
          entity_data[key] = value.get("value") if isinstance(value, dict) else value
      entities.append(entity_data)

    # Convert the list of entities to a DataFrame
    df = pd.DataFrame(entities)
    return df

  except requests.exceptions.RequestException as e:
    print(f"Error fetching data from Orion Context Broker: {e}")
    return pd.DataFrame()
  except ValueError as e:
    print(f"Error parsing JSON response: {e}")
    return pd.DataFrame()

In [7]:
# イベント情報
params = {
  "idPattern": ".*12354_event.*",
  "limit": 100,
}

access_token = get_token(client_id, client_secret)
data = request_data(access_token, params)
# display(data)

df = convert_to_dataframe(data)
display(df)

,id,type,capacity_notes,contact_phone_number,content_url,corporate_number,description,end_date,end_time,event_id,...,url,web_event,application_method,latitude,longitude,co_organizer,related_organization,access_method,building_name_or_other,meeting_reception_location
0,urn:ngsi-ld:12354_event:190004000000118556,12354_event,どぐたこ作り：16名（4回開催・各回4名）\r\n\r\nクイズラリー「考古福めぐり」の参加...,055-266-3881,https://www.pref.yamanashi.jp/event/kouko-hak/...,8000020190004,今回のイベントでは、縁起物にちなんだクイズラリー「考古福めぐり」をはじめ、考古博の土偶がモチ...,"[2025-01-02, 2025-01-03]","[00:00, 00:00]",190004000000118560,...,https://www.pref.yamanashi.jp/kouko-hak/index....,無,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,urn:ngsi-ld:12354_event:190004000000118278,12354_event,小・中学生の子どもとその保護者・16組（1月19日は午前の部・午後の部ともに8組）,055-266-3881,https://www.pref.yamanashi.jp/event/kouko-hak/...,8000020190004,山梨県は縄文王国と呼ばれるほど沢山の縄文時代の遺跡があり、各地の遺跡からは美しい文様や装飾が...,"[2025-01-19, 2025-02-02]","[00:00, 00:00]",190004000000118272,...,https://www.pref.yamanashi.jp/kouko-hak/index....,無,参加には事前にお申し込みが必要です。\r\n風土記の丘研修センターへお電話またはご来館の上、...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,urn:ngsi-ld:12354_event:190004000000118559,12354_event,対面開催約40名（先着順）\r\nオンライン開催約60名（先着順）\r\n※定員により受講で...,NaN,https://www.pref.yamanashi.jp/event/rosei-jin/...,8000020190004,【開催内容】\r\n〔第1部〕14:00～15:30\r\n ①講演 ～マッチングの大切さと...,2025-01-14,16:00,190004000000118560,...,https://www.pref.yamanashi.jp/rosei-jin/shogai...,有,%E4%BB%A5%E4%B8%8B%E3%81%AEURL%E3%82%88%E3%82%...,35.628855,138.571239,NaN,NaN,NaN,NaN,NaN
3,urn:ngsi-ld:12354_event:190004000000118123,12354_event,黒米もちとおしるこの配布はそれぞれ100名ほどを予定しています。\r\nなお、配布は餅つき体...,055-266-5854,https://www.pref.yamanashi.jp/event/kouko-hak/...,8000020190004,今回のイベントでは小正月行事として、赤い色をした古代米を古代より使われていたとされる竪杵（た...,2025-01-13,00:00,190004000000118112,...,https://www.sonekyuryo.jp/,無,事前の参加お申し込みは不要です。,NaN,NaN,共催：山梨県立考古博物館\r\n協力：山梨県立考古博物館協力会,共催：山梨県立考古博物館\r\n協力：山梨県立考古博物館協力会,NaN,NaN,NaN
4,urn:ngsi-ld:12354_event:190004000000118482,12354_event,NaN,055-266-3881,https://www.pref.yamanashi.jp/event/kouko-hak/...,8000020190004,今回のミニ展示では、山梨県内の遺跡から出土したヘビに関連する考古資料をご紹介するとともに、古...,2025-01-26,00:00,190004000000118496,...,https://www.pref.yamanashi.jp/kouko-hak/index....,無,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,urn:ngsi-ld:12354_event:190004000000112759,12354_event,先着600名（要事前申込 2月16日から受付開始）,NaN,https://www.pref.yamanashi.jp/event/bunka/shoc...,8000020190004,本イベントは山梨県と松竹株式会社が締結した「文化・芸術活動の振興等に関する連携基本協定」の...,2024-03-20,16:30,190004000000112768,...,https://form.run/@yamanashikabuki/,無,次のいずれかの方法でお申し込みください。\r\n1.YCC県民文化ホールのHPのイベント案内...,NaN,NaN,制作 松竹株式会社,制作 松竹株式会社,NaN,NaN,NaN
96,urn:ngsi-ld:12354_event:190004000000112757,12354_event,NaN,NaN,https://www.pref.yamanashi.jp/event/rosei-jin/...,8000020190004,NaN,2024-03-06,15:30,190004000000112768,...,https%3A//apply.e-tumo.jp/pref-yamanashi-u/off...,無,現在、参加者を募集しております。\r\nお申込みは、ページ下部関連リンクの「参加者用申込みフ...,NaN,NaN,共催：山梨労働局／ハローワーク、甲府商工会議所\r\n後援：山梨県中小企業団体中央会、山梨県...,共催：山梨労働局／ハローワーク、甲府商工会議所\r\n後援：山梨県中小企業団体中央会、山梨県...,NaN,NaN,NaN
97,urn:ngsi-ld:12354_event:190004000000112758,12354_event,NaN,NaN,https://www.pref.yamanashi.jp/event/rosei-jin/...,8000020190004,NaN,2024-03-16,15:30,190004000000112768,...,https%3A//apply.e-tumo.jp/pref-yamanashi-u/off...,無,現在、参加者を募集しております。\r\nお申込みは、ページ下部関連リンクの「参加者用申込みフ...,NaN,NaN,共催：山梨労働局／ハローワーク、甲府商工会議所\r\n後援：山梨県中小企業団体中央会、山梨県...,共催：山梨労働局／ハローワーク、甲府商工会議所\r\n後援：山梨県中小企業団体中央会、山梨県...,NaN,NaN,NaN
98,urn:ngsi-ld:12354_event:190004000000112584,12354_event,NaN,NaN,https://www.pref.yamanashi.jp/chisui/ryuikichi...,8000020190004,NaN,2024-02-21,16:00,190004000000112576,...,NaN,NaN,チラシに記載の申込みフォームからお申込みください。,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# 市町村別国調人口・世帯数(2020年)

params = {
    "idPattern": ".*EDAA02000.*",
    "type": "EDAA02000-2020-10-01",
    "limit": 100,
}

access_token = get_token(client_id, client_secret)
data = request_data(access_token, params)
df = convert_to_dataframe(data)
display(df)

,id,type,acquisition_date,area_name,female,male,number_of_households,total_number
0,urn:ngsi-ld:EDAA02000-2020-10-01:1,EDAA02000-2020-10-01,2020-10-01T00:00:00.000Z,県合計,412665,397309,338853,809974
1,urn:ngsi-ld:EDAA02000-2020-10-01:2,EDAA02000-2020-10-01,2020-10-01T00:00:00.000Z,市部計,352847,338198,290327,691045
2,urn:ngsi-ld:EDAA02000-2020-10-01:3,EDAA02000-2020-10-01,2020-10-01T00:00:00.000Z,郡部計,59818,59111,48526,118929
3,urn:ngsi-ld:EDAA02000-2020-10-01:4,EDAA02000-2020-10-01,2020-10-01T00:00:00.000Z,甲府市,96741,92850,87019,189591
4,urn:ngsi-ld:EDAA02000-2020-10-01:5,EDAA02000-2020-10-01,2020-10-01T00:00:00.000Z,富士吉田市,23885,22645,18337,46530
5,urn:ngsi-ld:EDAA02000-2020-10-01:6,EDAA02000-2020-10-01,2020-10-01T00:00:00.000Z,都留市,16067,14949,14044,31016
6,urn:ngsi-ld:EDAA02000-2020-10-01:7,EDAA02000-2020-10-01,2020-10-01T00:00:00.000Z,山梨市,17354,16081,13001,33435
7,urn:ngsi-ld:EDAA02000-2020-10-01:8,EDAA02000-2020-10-01,2020-10-01T00:00:00.000Z,大月市,11588,10924,9254,22512
8,urn:ngsi-ld:EDAA02000-2020-10-01:9,EDAA02000-2020-10-01,2020-10-01T00:00:00.000Z,韮崎市,14567,14500,11552,29067
9,urn:ngsi-ld:EDAA02000-2020-10-01:10,EDAA02000-2020-10-01,2020-10-01T00:00:00.000Z,南アルプス市,35295,34164,26059,69459


In [9]:
# 市町村別国調人口・世帯数(甲府市)

params = {
    "idPattern": ".*EDAA02000.*",
    "q": "area_name==甲府市",    # クエリ条件
    "orderBy": "acquisition_date",    # ソート項目
    "attrs": "male,female,total_number",    # 取得したい属性のみ指定
    "limit": 100,
}

access_token = get_token(client_id, client_secret)
data = request_data(access_token, params)
df = convert_to_dataframe(data)
display(df)

,id,type,male,female,total_number
0,urn:ngsi-ld:EDAA02000-1920-10-01:4,EDAA02000-1920-10-01,44576,44233,88809
1,urn:ngsi-ld:EDAA02000-1925-10-01:4,EDAA02000-1925-10-01,52216,50884,103100
2,urn:ngsi-ld:EDAA02000-1930-10-01:4,EDAA02000-1930-10-01,58636,58536,117172
3,urn:ngsi-ld:EDAA02000-1935-10-01:4,EDAA02000-1935-10-01,61108,62135,123243
4,urn:ngsi-ld:EDAA02000-1940-10-01:4,EDAA02000-1940-10-01,59182,63124,122306
5,urn:ngsi-ld:EDAA02000-1947-10-01:4,EDAA02000-1947-10-01,60173,64967,125140
6,urn:ngsi-ld:EDAA02000-1950-10-01:4,EDAA02000-1950-10-01,67401,71636,139037
7,urn:ngsi-ld:EDAA02000-1955-10-01:4,EDAA02000-1955-10-01,74649,79852,154501
8,urn:ngsi-ld:EDAA02000-1960-10-01:4,EDAA02000-1960-10-01,77416,83547,160963
9,urn:ngsi-ld:EDAA02000-1965-10-01:4,EDAA02000-1965-10-01,82976,89481,172457
